# Подготовка новостей для библиотеки gensim
[gensim](https://radimrehurek.com/gensim/tut1.html)

# Модель новости

In [31]:
import json
import numpy as np
from pymystem3 import Mystem
import re
from collections import defaultdict
from gensim import corpora

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

class News:
    def __init__(self, id, date, annotation, title, content, url):
        self.id = id
        self.date = date
        self.annotation = annotation
        self.title = title
        self.content = content
        self.url = url
    
    @classmethod
    def from_json(cls, json_str):
        json_dict = json.loads(json_str)
        return cls(**json_dict)

# Загружаем новости

In [9]:
news = []
with open('/data/kasandra/news-4965.json', encoding="utf8") as f:
    for line in f:
        news.append(News.from_json(line))
        
news_contenst = list(map(lambda w: w.content, news))

list(map(lambda w: w.title, news[:5]))

['На Алтае заработала первая в РФ система контроля космического пространства',
 'ЛНР: Действия украинских диверсантов — одна из версий покушения на Плотницкого',
 'Bild не публикует результаты России в медальном зачёте Олимпиады',
 'Глава ЛНР связал покушение на свою жизнь с украинскими спецслужбами',
 'Мэра города в США арестовали за попытку расплатиться наркотиками за секс с мужчинами']

# Загружаем стоп слова

In [7]:
stopwords = []

with open('../main/python/res/stopwords.txt', mode="r", encoding="utf8") as file:
    for line in file:
        stopwords.append(line.replace('\n', ''))
        
stopwords[:5]

['без', 'более', 'больше', 'будет', 'будто']

# Подключаем Yandex MyStem для нормализации текста

In [11]:
mystem = Mystem(mystem_bin='/data/mystem/mystem',
                entire_input=False)

# Оставляем только русские слова
r = re.compile('^[А-ЯЙа-яй]*$')

def lemmatize(text):
    return list(filter(r.match, mystem.lemmatize(text)))

## Токенизируем текст и удаляем стоп-слова

In [29]:
# Удаляем стоп-слова
texts = [[word for word in lemmatize(document) if word not in stopwords]
         for document in news_contenst]

# Удаляем слова которые встречаются только один раз
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1
        
texts = [[token for token in text if frequency[token] > 1]
         for text in texts]

In [30]:
print(texts[11])
news_contenst[11]

['британский', 'служба', 'береговой', 'охрана', 'заниматься', 'координация', 'поиск', 'легкий', 'самолет', 'совершать', 'аварийный', 'посадка', 'вода', 'деревня', 'рай', 'харбор', 'графство', 'восточный', 'сообщать', 'тасс', 'тревога', 'подымать', 'мск', 'сообщение', 'указывать', 'борт', 'находиться', 'человек', 'самолет', 'пилот', 'обнаруживать', 'нынешний', 'поиск', 'продлиться', 'наступление', 'темный', 'время', 'сутки', 'подчеркивать', 'представитель', 'служба', 'береговой', 'охрана']


'Британская Служба береговой охраны «занимается координацией поиска лёгкого самолёт, который совершил аварийную посадку на воду в полумиле от деревни Рай-Харбор в графстве Восточный Сассекс», сообщает ТАСС. «Тревога была поднята в 17:05 (19:05 мск. —\xa0RT) и сообщения указывают на то, что на борту находился один человек. Самолёт и пилот не были обнаружены, и нынешние поиски продлятся до наступления темного времени суток», —\xa0подчеркнул представитель Службы береговой охраны.\xa0'

## Выгружаем получившиеся данные

In [34]:
dictionary = corpora.Dictionary(texts)
dictionary.save('/data/kasandra/gensim/news-4965.dict')

corpus = [dictionary.doc2bow(text) for text in texts]
corpora.MmCorpus.serialize('/data/kasandra/gensim/news-4965.mm', corpus)

In [36]:
#print(dictionary.token2id)